In [1]:
import torch
from torch.utils.data import DataLoader
from torchmetrics import MeanAbsoluteError, MeanSquaredError
import numpy as np
from data.perovskite_dataset import PerovskiteDataset1d
from data.augmentations.perov_1d import normalize as normalize_1d

from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.decomposition import NMF, PCA
import xgboost as xgb

# Tree based / Boosting

## Random Forest

In [2]:
data_dir = '/home/s522r/Desktop/perovskite/new_data/2021_KIT_PerovskiteDeposition/preprocessed'

val_mse_folds = []
val_mae_folds = []

for fold_nb in range(5):
    print('Fold: ', fold_nb)
    
    train_mean, train_std = PerovskiteDataset1d(data_dir=data_dir, transform=None, fold=fold_nb,
                                                                  split='train', label='PCE_mean').get_stats()
    
    trainset = PerovskiteDataset1d(data_dir=data_dir,
                                           transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                           split='train', label='PCE_mean', val=False)
    trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)
    
    valset = PerovskiteDataset1d(data_dir=data_dir,
                                         transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                         split='train', label='PCE_mean', val=True)
    valloader = DataLoader(valset, batch_size=len(valset), shuffle=False)
    
    for timeseries, label in trainloader:
        
        time = timeseries.numpy().reshape(-1,4*719)#[:,3,:]
        
        rfr = RandomForestRegressor(n_estimators=100)
        rfr.fit(time, label.numpy())
        
    for timeseries, label in valloader:
        
        time = timeseries.numpy().reshape(-1,4*719)#[:,3,:]
        scores = rfr.predict(time)
        
        mse = MeanSquaredError()
        mae = MeanAbsoluteError()
        
        val_mse = mse(torch.from_numpy(scores), label)
        print('MSE: ',val_mse)
        val_mse_folds.append(val_mse)
        
        val_mae = mae(torch.from_numpy(scores), label)
        print('MAE:',val_mae)
        val_mae_folds.append(val_mae)
    
    
print('Mean MSE: ', np.mean(val_mse_folds))
print('Mean MAE: ', np.mean(val_mae_folds))

Fold:  0
MSE:  tensor(8.5245)
MAE: tensor(2.1031)
Fold:  1
MSE:  tensor(1.9241)
MAE: tensor(1.0332)
Fold:  2
MSE:  tensor(2.6146)
MAE: tensor(1.1550)
Fold:  3
MSE:  tensor(4.8551)
MAE: tensor(1.4217)
Fold:  4
MSE:  tensor(4.4215)
MAE: tensor(1.5391)
Mean MSE:  4.467955
Mean MAE:  1.4504067


## Ada Boost

In [3]:
data_dir = '/home/s522r/Desktop/perovskite/new_data/2021_KIT_PerovskiteDeposition/preprocessed'

val_mse_folds = []
val_mae_folds = []

for fold_nb in range(5):
    print('Fold: ', fold_nb)
    
    train_mean, train_std = PerovskiteDataset1d(data_dir=data_dir, transform=None, fold=fold_nb,
                                                                  split='train', label='PCE_mean').get_stats()
    
    trainset = PerovskiteDataset1d(data_dir=data_dir,
                                           transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                           split='train', label='PCE_mean', val=False)
    trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)
    
    valset = PerovskiteDataset1d(data_dir=data_dir,
                                         transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                         split='train', label='PCE_mean', val=True)
    valloader = DataLoader(valset, batch_size=len(valset), shuffle=False)
    
    for timeseries, label in trainloader:
        
        time = timeseries.numpy().reshape(-1,4*719)#[:,3,:]
        
        abr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=5))
        abr.fit(time, label.numpy())
        
    for timeseries, label in valloader:
        
        time = timeseries.numpy().reshape(-1,4*719)#[:,3,:]
        scores = abr.predict(time)
        
        mse = MeanSquaredError()
        mae = MeanAbsoluteError()
        
        val_mse = mse(torch.from_numpy(scores), label)
        print('MSE: ',val_mse)
        val_mse_folds.append(val_mse)
        
        val_mae = mae(torch.from_numpy(scores), label)
        print('MAE:',val_mae)
        val_mae_folds.append(val_mae)
    
    
print('Mean MSE: ', np.mean(val_mse_folds))
print('Mean MAE: ', np.mean(val_mae_folds))

Fold:  0
MSE:  tensor(8.2357)
MAE: tensor(2.0782)
Fold:  1
MSE:  tensor(2.0974)
MAE: tensor(1.0977)
Fold:  2
MSE:  tensor(2.9211)
MAE: tensor(1.2231)
Fold:  3
MSE:  tensor(4.8320)
MAE: tensor(1.4653)
Fold:  4
MSE:  tensor(4.0939)
MAE: tensor(1.4298)
Mean MSE:  4.435995
Mean MAE:  1.4588273


## XGBoost

In [4]:
data_dir = '/home/s522r/Desktop/perovskite/new_data/2021_KIT_PerovskiteDeposition/preprocessed'

val_mse_folds = []
val_mae_folds = []

for fold_nb in range(5):
    print('Fold: ', fold_nb)
    
    train_mean, train_std = PerovskiteDataset1d(data_dir=data_dir, transform=None, fold=fold_nb,
                                                                  split='train', label='PCE_mean').get_stats()
    
    trainset = PerovskiteDataset1d(data_dir=data_dir,
                                           transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                           split='train', label='PCE_mean', val=False)
    trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)
    
    valset = PerovskiteDataset1d(data_dir=data_dir,
                                         transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                         split='train', label='PCE_mean', val=True)
    valloader = DataLoader(valset, batch_size=len(valset), shuffle=False)
    
    for timeseries, label in trainloader:
        
        time = timeseries.numpy().reshape(-1,4*719)#[:,3,:]
        
        xgbr = xgb.XGBRegressor(n_estimators = 100, tree_method="gpu_hist", n_jobs=20, max_depth=6, learning_rate=0.3,
                               booster='gbtree', num_parallel_tree=1, objective='reg:squarederror')
        xgbr.fit(time, label.numpy())
        
    for timeseries, label in valloader:
        
        time = timeseries.numpy().reshape(-1,4*719)#[:,3,:]
        scores = xgbr.predict(time)
        
        mse = MeanSquaredError()
        mae = MeanAbsoluteError()
        
        val_mse = mse(torch.from_numpy(scores), label)
        print('MSE: ',val_mse)
        val_mse_folds.append(val_mse)
        
        val_mae = mae(torch.from_numpy(scores), label)
        print('MAE:',val_mae)
        val_mae_folds.append(val_mae)
    
    
print('Mean MSE: ', np.mean(val_mse_folds))
print('Mean MAE: ', np.mean(val_mae_folds))

Fold:  0
MSE:  tensor(9.3076)
MAE: tensor(2.2595)
Fold:  1
MSE:  tensor(2.0291)
MAE: tensor(1.1038)
Fold:  2
MSE:  tensor(3.0052)
MAE: tensor(1.2740)
Fold:  3
MSE:  tensor(5.6911)
MAE: tensor(1.5379)
Fold:  4
MSE:  tensor(5.9769)
MAE: tensor(1.7618)
Mean MSE:  5.201976
Mean MAE:  1.5874028


## Gradient Boosting

In [5]:
data_dir = '/home/s522r/Desktop/perovskite/new_data/2021_KIT_PerovskiteDeposition/preprocessed'

val_mse_folds = []
val_mae_folds = []

for fold_nb in range(5):
    print('Fold: ', fold_nb)
    
    train_mean, train_std = PerovskiteDataset1d(data_dir=data_dir, transform=None, fold=fold_nb,
                                                                  split='train', label='PCE_mean').get_stats()
    
    trainset = PerovskiteDataset1d(data_dir=data_dir,
                                           transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                           split='train', label='PCE_mean', val=False)
    trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)
    
    valset = PerovskiteDataset1d(data_dir=data_dir,
                                         transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                         split='train', label='PCE_mean', val=True)
    valloader = DataLoader(valset, batch_size=len(valset), shuffle=False)
    
    for timeseries, label in trainloader:
        
        time = timeseries.numpy().reshape(-1,4*719)#[:,3,:]
        
        gbr = GradientBoostingRegressor()
        gbr.fit(time, label.numpy())
        
    for timeseries, label in valloader:
        
        time = timeseries.numpy().reshape(-1,4*719)#[:,3,:]
        scores = gbr.predict(time)
        
        mse = MeanSquaredError()
        mae = MeanAbsoluteError()
        
        val_mse = mse(torch.from_numpy(scores), label)
        print('MSE: ',val_mse)
        val_mse_folds.append(val_mse)
        
        val_mae = mae(torch.from_numpy(scores), label)
        print('MAE:',val_mae)
        val_mae_folds.append(val_mae)
    
    
print('Mean MSE: ', np.mean(val_mse_folds))
print('Mean MAE: ', np.mean(val_mae_folds))

Fold:  0
MSE:  tensor(8.2781)
MAE: tensor(2.0419)
Fold:  1
MSE:  tensor(2.1432)
MAE: tensor(1.0562)
Fold:  2
MSE:  tensor(3.1122)
MAE: tensor(1.2753)
Fold:  3
MSE:  tensor(4.9790)
MAE: tensor(1.3992)
Fold:  4
MSE:  tensor(4.7386)
MAE: tensor(1.5267)
Mean MSE:  4.6502275
Mean MAE:  1.4598696


## Decision Tree

In [6]:
data_dir = '/home/s522r/Desktop/perovskite/new_data/2021_KIT_PerovskiteDeposition/preprocessed'

val_mse_folds = []
val_mae_folds = []

for fold_nb in range(5):
    print('Fold: ', fold_nb)
    
    train_mean, train_std = PerovskiteDataset1d(data_dir=data_dir, transform=None, fold=fold_nb,
                                                                  split='train', label='PCE_mean').get_stats()
    
    trainset = PerovskiteDataset1d(data_dir=data_dir,
                                           transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                           split='train', label='PCE_mean', val=False)
    trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)
    
    valset = PerovskiteDataset1d(data_dir=data_dir,
                                         transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                         split='train', label='PCE_mean', val=True)
    valloader = DataLoader(valset, batch_size=len(valset), shuffle=False)
    
    for timeseries, label in trainloader:
        
        time = timeseries.numpy().reshape(-1,4*719)#[:,3,:]
        
        dtr = DecisionTreeRegressor(max_depth=5)
        dtr.fit(time, label.numpy())
        
    for timeseries, label in valloader:
        
        time = timeseries.numpy().reshape(-1,4*719)#[:,3,:]
        scores = dtr.predict(time)
        
        mse = MeanSquaredError()
        mae = MeanAbsoluteError()
        
        val_mse = mse(torch.from_numpy(scores), label)
        print('MSE: ',val_mse)
        val_mse_folds.append(val_mse)
        
        val_mae = mae(torch.from_numpy(scores), label)
        print('MAE:',val_mae)
        val_mae_folds.append(val_mae)
    
    
print('Mean MSE: ', np.mean(val_mse_folds))
print('Mean MAE: ', np.mean(val_mae_folds))

Fold:  0
MSE:  tensor(9.0047)
MAE: tensor(2.1085)
Fold:  1
MSE:  tensor(2.8880)
MAE: tensor(1.2817)
Fold:  2
MSE:  tensor(3.8511)
MAE: tensor(1.4510)
Fold:  3
MSE:  tensor(7.1705)
MAE: tensor(1.7495)
Fold:  4
MSE:  tensor(8.6616)
MAE: tensor(1.8284)
Mean MSE:  6.3151636
Mean MAE:  1.6838017


# Clustering / Nearest Neighbors

## KNN

In [7]:
data_dir = '/home/s522r/Desktop/perovskite/new_data/2021_KIT_PerovskiteDeposition/preprocessed'

val_mse_folds = []
val_mae_folds = []

for fold_nb in range(5):
    print('Fold: ', fold_nb)
    
    train_mean, train_std = PerovskiteDataset1d(data_dir=data_dir, transform=None, fold=fold_nb,
                                                                  split='train', label='PCE_mean').get_stats()
    
    trainset = PerovskiteDataset1d(data_dir=data_dir,
                                           transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                           split='train', label='PCE_mean', val=False)
    trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)
    
    valset = PerovskiteDataset1d(data_dir=data_dir,
                                         transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                         split='train', label='PCE_mean', val=True)
    valloader = DataLoader(valset, batch_size=len(valset), shuffle=False)
    
    for timeseries, label in trainloader:
        
        time = timeseries.numpy().reshape(-1,4*719)#[:,3,:]
        
        knn = KNeighborsRegressor(n_neighbors=20)
        knn.fit(time, label.numpy())
        
    for timeseries, label in valloader:
        
        time = timeseries.numpy().reshape(-1,4*719)#[:,3,:]
        scores = knn.predict(time)
        
        mse = MeanSquaredError()
        mae = MeanAbsoluteError()
        
        val_mse = mse(torch.from_numpy(scores), label)
        print('MSE: ',val_mse)
        val_mse_folds.append(val_mse)
        
        val_mae = mae(torch.from_numpy(scores), label)
        print('MAE:',val_mae)
        val_mae_folds.append(val_mae)
    
    
print('Mean MSE: ', np.mean(val_mse_folds))
print('Mean MAE: ', np.mean(val_mae_folds))

Fold:  0
MSE:  tensor(8.8568)
MAE: tensor(2.2043)
Fold:  1
MSE:  tensor(1.8798)
MAE: tensor(1.0235)
Fold:  2
MSE:  tensor(4.1541)
MAE: tensor(1.5253)
Fold:  3
MSE:  tensor(6.0270)
MAE: tensor(1.6227)
Fold:  4
MSE:  tensor(5.4559)
MAE: tensor(1.6094)
Mean MSE:  5.274737
Mean MAE:  1.5970377


## KMeans

In [8]:
data_dir = '/home/s522r/Desktop/perovskite/new_data/2021_KIT_PerovskiteDeposition/preprocessed'

val_mse_folds = []
val_mae_folds = []

for fold_nb in range(5):
    print('Fold: ', fold_nb)
    
    train_mean, train_std = PerovskiteDataset1d(data_dir=data_dir, transform=None, fold=fold_nb,
                                                                  split='train', label='PCE_mean').get_stats()
    
    trainset = PerovskiteDataset1d(data_dir=data_dir,
                                           transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                           split='train', label='PCE_mean', val=False)
    trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)
    
    valset = PerovskiteDataset1d(data_dir=data_dir,
                                         transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                         split='train', label='PCE_mean', val=True)
    valloader = DataLoader(valset, batch_size=len(valset), shuffle=False)
    
    for timeseries, label in trainloader:
        
        nb_clusters = 10
        
        time = timeseries.numpy().reshape(-1,4*719)
        label = label.numpy()
        
        kmeans = KMeans(nb_clusters)
        clusters = kmeans.fit_predict(time)
        
        cluster_means = []
        for i in range(nb_clusters):
            cluster_means.append(label[(clusters==i)].mean())
        
    for timeseries, label in valloader:
        
        time = timeseries.numpy().reshape(-1,4*719)
        
        val_clusters = kmeans.predict(time)
        scores = np.array([cluster_means[i] for i in val_clusters])
        
        mse = MeanSquaredError()
        mae = MeanAbsoluteError()
        
        val_mse = mse(torch.from_numpy(scores), label)
        print('MSE: ',val_mse)
        val_mse_folds.append(val_mse)
        
        val_mae = mae(torch.from_numpy(scores), label)
        print('MAE:',val_mae)
        val_mae_folds.append(val_mae)
        
    
print('Mean MSE: ', np.mean(val_mse_folds))
print('Mean MAE: ', np.mean(val_mae_folds))

Fold:  0
MSE:  tensor(8.6436)
MAE: tensor(2.2130)
Fold:  1
MSE:  tensor(2.4791)
MAE: tensor(1.2011)
Fold:  2
MSE:  tensor(5.6935)
MAE: tensor(1.8630)
Fold:  3
MSE:  tensor(7.5174)
MAE: tensor(1.9538)
Fold:  4
MSE:  tensor(7.0760)
MAE: tensor(1.8818)
Mean MSE:  6.281933
Mean MAE:  1.8225473


## PCA + KMeans

In [9]:
data_dir = '/home/s522r/Desktop/perovskite/new_data/2021_KIT_PerovskiteDeposition/preprocessed'

val_mse_folds = []
val_mae_folds = []

for fold_nb in range(5):
    print('Fold: ', fold_nb)
    
    train_mean, train_std = PerovskiteDataset1d(data_dir=data_dir, transform=None, fold=fold_nb,
                                                                  split='train', label='PCE_mean').get_stats()
    
    trainset = PerovskiteDataset1d(data_dir=data_dir,
                                           transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                           split='train', label='PCE_mean', val=False)
    trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)
    
    valset = PerovskiteDataset1d(data_dir=data_dir,
                                         transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                         split='train', label='PCE_mean', val=True)
    valloader = DataLoader(valset, batch_size=len(valset), shuffle=False)
    
    for timeseries, label in trainloader:
        
        nb_clusters = 10
        
        time = timeseries.numpy().reshape(-1,4*719)
        pca = PCA(n_components=5)
        
        label = label.numpy()
        
        kmeans = KMeans(nb_clusters)
        clusters = kmeans.fit_predict(pca.fit_transform(time))
        
        cluster_means = []
        for i in range(nb_clusters):
            cluster_means.append(label[(clusters==i)].mean())
        
    for timeseries, label in valloader:
        
        time = timeseries.numpy().reshape(-1,4*719)
        
        val_clusters = kmeans.predict(pca.transform(time))
        scores = np.array([cluster_means[i] for i in val_clusters])
        
        mse = MeanSquaredError()
        mae = MeanAbsoluteError()
        
        val_mse = mse(torch.from_numpy(scores), label)
        print('MSE: ',val_mse)
        val_mse_folds.append(val_mse)
        
        val_mae = mae(torch.from_numpy(scores), label)
        print('MAE:',val_mae)
        val_mae_folds.append(val_mae)
        
    
print('Mean MSE: ', np.mean(val_mse_folds))
print('Mean MAE: ', np.mean(val_mae_folds))

Fold:  0
MSE:  tensor(8.5856)
MAE: tensor(2.2128)
Fold:  1
MSE:  tensor(2.6763)
MAE: tensor(1.2359)
Fold:  2
MSE:  tensor(5.7514)
MAE: tensor(1.9025)
Fold:  3
MSE:  tensor(7.6592)
MAE: tensor(1.9545)
Fold:  4
MSE:  tensor(6.8288)
MAE: tensor(1.8092)
Mean MSE:  6.300251
Mean MAE:  1.8229691


## NMF + KMeans

In [10]:
data_dir = '/home/s522r/Desktop/perovskite/new_data/2021_KIT_PerovskiteDeposition/preprocessed'

val_mse_folds = []
val_mae_folds = []

for fold_nb in range(5):
    print('Fold: ', fold_nb)
    
    train_mean, train_std = PerovskiteDataset1d(data_dir=data_dir, transform=None, fold=fold_nb,
                                                                  split='train', label='PCE_mean').get_stats()
    
    trainset = PerovskiteDataset1d(data_dir=data_dir,
                                           transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                           split='train', label='PCE_mean', val=False)
    trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)
    
    valset = PerovskiteDataset1d(data_dir=data_dir,
                                         transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                         split='train', label='PCE_mean', val=True)
    valloader = DataLoader(valset, batch_size=len(valset), shuffle=False)
    
    for timeseries, label in trainloader:
        
        nb_clusters = 10
        
        time = timeseries.numpy().reshape(-1,4*719)
        nmf = NMF(n_components=5, init='nndsvda', max_iter=600)
        
        label = label.numpy()
        
        kmeans = KMeans(nb_clusters)
        clusters = kmeans.fit_predict(nmf.fit_transform(time+2))
        
        cluster_means = []
        for i in range(nb_clusters):
            cluster_means.append(label[(clusters==i)].mean())
        
    for timeseries, label in valloader:
        
        time = timeseries.numpy().reshape(-1,4*719)
        
        val_clusters = kmeans.predict(nmf.transform(time+2))
        scores = np.array([cluster_means[i] for i in val_clusters])
        
        mse = MeanSquaredError()
        mae = MeanAbsoluteError()
        
        val_mse = mse(torch.from_numpy(scores), label)
        print('MSE: ',val_mse)
        val_mse_folds.append(val_mse)
        
        val_mae = mae(torch.from_numpy(scores), label)
        print('MAE:',val_mae)
        val_mae_folds.append(val_mae)
        
    
print('Mean MSE: ', np.mean(val_mse_folds))
print('Mean MAE: ', np.mean(val_mae_folds))

Fold:  0
MSE:  tensor(10.3261)
MAE: tensor(2.4038)
Fold:  1
MSE:  tensor(3.0688)
MAE: tensor(1.3667)
Fold:  2
MSE:  tensor(4.0055)
MAE: tensor(1.4702)
Fold:  3
MSE:  tensor(6.8677)
MAE: tensor(1.8194)
Fold:  4
MSE:  tensor(6.0449)
MAE: tensor(1.7059)
Mean MSE:  6.0626154
Mean MAE:  1.7532008


# Simple Mean

## Mean

In [11]:
data_dir = '/home/s522r/Desktop/perovskite/new_data/2021_KIT_PerovskiteDeposition/preprocessed'

val_mse_folds = []
val_mae_folds = []

for fold_nb in range(5):
    print('Fold: ', fold_nb)
    
    train_mean, train_std = PerovskiteDataset1d(data_dir=data_dir, transform=None, fold=fold_nb,
                                                                  split='train', label='PCE_mean').get_stats()
    
    trainset = PerovskiteDataset1d(data_dir=data_dir,
                                           transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                           split='train', label='PCE_mean', val=False)
    trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)
    
    valset = PerovskiteDataset1d(data_dir=data_dir,
                                         transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                         split='train', label='PCE_mean', val=True)
    valloader = DataLoader(valset, batch_size=len(valset), shuffle=False)
    
    for timeseries, label in trainloader:
        
        mean = label.numpy().mean()
        
    for timeseries, label in valloader:
        
        scores = np.array([mean]*len(label))
        
        mse = MeanSquaredError()
        mae = MeanAbsoluteError()
        
        val_mse = mse(torch.from_numpy(scores), label)
        print('MSE: ',val_mse)
        val_mse_folds.append(val_mse)
        
        val_mae = mae(torch.from_numpy(scores), label)
        print('MAE:',val_mae)
        val_mae_folds.append(val_mae)
    
    
print('Mean MSE: ', np.mean(val_mse_folds))
print('Mean MAE: ', np.mean(val_mae_folds))

Fold:  0
MSE:  tensor(8.3835)
MAE: tensor(2.1899)
Fold:  1
MSE:  tensor(4.2631)
MAE: tensor(1.6608)
Fold:  2
MSE:  tensor(5.1630)
MAE: tensor(1.7084)
Fold:  3
MSE:  tensor(8.1214)
MAE: tensor(2.0111)
Fold:  4
MSE:  tensor(7.6820)
MAE: tensor(1.9131)
Mean MSE:  6.722593
Mean MAE:  1.8966551
